### Import the libraries

In [5]:
import alfalfa_client.alfalfa_client as ac
import alfalfa_client.historian as ah

import os, requests, datetime

In [6]:

client = ac.AlfalfaClient()
historian = ah.Historian()


### Submit your testcase. 
Each time you submit a testcase, the function returns its ID.

For more information: [Alfalfa's Wiki](https://github.com/NREL/alfalfa/wiki/OpenStudio-Model-Setup)

In [7]:
zippath = "examples/with_ep2brick.zip"
#zippath = "examples/primaryschool.zip"
ref = client.submit(os.path.join('..', zippath))

Desired status: ready		Current status: preprocessing
Desired status: ready		Current status: ready


### Start a testcase
You need its ID, a start and end time. Here, we use an external clock to control the simulation. The external clock can ignore the end time.

In [8]:

start = datetime.datetime(2022, 1, 3, 7, 0, 0)
end = (start + datetime.timedelta(days=365)).strftime('%Y-%m-%d %H:%M:%S')
start = start.strftime('%Y-%m-%d %H:%M:%S')
inp = client.start(ref, start_datetime=start, end_datetime=end, external_clock=True)

Desired status: running		Current status: ready
Desired status: running		Current status: started
Desired status: running		Current status: running


### Get testcase information

See all the testcase outputs

In [ ]:
sensors = client.get_outputs(ref)
sensors

See all the testcase inputs

In [ ]:
points = client.get_inputs(ref)
points

### Override controls

We can update the control vector at each timestep. The u vector is a dict.

In [10]:
steps = 10
commands = [25 for i in range(steps)]

controlledzone = 'mult_class_2_pod_2_zn_1_flr_1 zn'
controlledcomponent = 'mult_class_2_pod_2_zn_1_flr_1 zn'
controlledvar = 'Heating_Temperature'
controlledsen = 'Zone Air Temperature'


for c in commands:
    u = {
    f"{controlledzone.replace(' ', '_')}_{controlledvar}_Setpoint" : c
    }
    response = client.set_inputs(ref, u)
    client.advance(ref)
    res = client.get_outputs(ref)
    ctime = client.get_sim_time(ref)
    zonetemp = res[f"{controlledzone} Zone Air Temperature Sensor"]
    curval = res[f"{controlledcomponent} {controlledsen} Sensor"]
    controlaction = res[f"{controlledzone.replace(' ', '_')}_{controlledvar}_Setpoint_Value"]
    enableval = res[f"{controlledzone.replace(' ', '_')}_{controlledvar}_Setpoint_Enable_Value"]
    print(f"\tTime: {ctime}\n\tZone temperature: {zonetemp}\Current: {curval}\n\tRequested: {controlaction}\tEnable: {enableval}")

[{'name': 'mult_class_2_pod_2_zn_1_flr_1_zn_Heating_Temperature_Setpoint', 'value': 25}]
	Time: 2022-01-03 07:11:00
	Zone temperature: 20.00014886760937\Current: 20.00014886760937
	Requested: 25	Enable: 1
[{'name': 'mult_class_2_pod_2_zn_1_flr_1_zn_Heating_Temperature_Setpoint', 'value': 25}]
	Time: 2022-01-03 07:11:00
	Zone temperature: 20.00014886760937\Current: 20.00014886760937
	Requested: 25	Enable: 1
[{'name': 'mult_class_2_pod_2_zn_1_flr_1_zn_Heating_Temperature_Setpoint', 'value': 25}]
	Time: 2022-01-03 07:11:00
	Zone temperature: 20.00014886760937\Current: 20.00014886760937
	Requested: 25	Enable: 1
[{'name': 'mult_class_2_pod_2_zn_1_flr_1_zn_Heating_Temperature_Setpoint', 'value': 25}]
	Time: 2022-01-03 07:11:00
	Zone temperature: 20.00014886760937\Current: 20.00014886760937
	Requested: 25	Enable: 1
[{'name': 'mult_class_2_pod_2_zn_1_flr_1_zn_Heating_Temperature_Setpoint', 'value': 25}]
	Time: 2022-01-03 07:11:00
	Zone temperature: 20.00014886760937\Current: 20.00014886760937


In [ ]:
client.stop(ref)